In [1]:
using DataFrames
using CSV

In [2]:
function mean(x)
    sum(x)/length(x)
end

mean (generic function with 1 method)

# Load MPSGE.jl Solutions

In [6]:
file_name = "FullReportCounter"

replace_dict = Dict(
    "Y" => [r"^Y\[\(:(\w{3}),\)]", s"\1"],
    "A" => [r"^A\[\(:(\w{3}),\)]", s"\1"],
    "MS" => [r"^MS\[\(:(\w{3}),\)]", s"\1"],
    "PA" => [r"^PA\[\(:(\w{3}),\)]", s"\1"],
    "PY" => [r"^PY\[\(:(\w{3}),\)]", s"\1"],
    "PVA" => [r"^PVA\[\(:(\w{6,7}),\)]", s"\1"],
    "PM" => [r"^PM\[\(:(\w{3}),\)]", s"\1"],
    "PFX" => [r"^PFX", s""],
    "RA" => [r"^RA", s""],
    );

d = DataFrame(CSV.File("./Results/"*file_name*".csv",stringtype=String)) |>
    x -> transform(x, :margin => (y -> ifelse.(y.==".","0",y)) => :margin) |>
    x -> transform(x, :margin => (y -> parse.(Float64,y))=>:margin);


df_mpsge = DataFrame()

for (var,(reg,sub)) in replace_dict
    df_ = d |>
        x-> filter(:var_name => y -> occursin.(reg ,y),x) |>
        x -> transform(x,
            :var_name => (y -> replace.(y, reg => sub)) => :domain,
            :var_name => (y -> var) => :var
        ) |>
        x -> select(x, [:var_name,:var,:domain,:value,:margin])
    
    df_mpsge = vcat(df_mpsge,df_)
end

df_mpsge = df_mpsge |>
    x -> select(x, [:var, :domain, :value])

Row,var,domain,value
,String,String,Float64
1,Y,ppd,1.0188
2,Y,res,1.03916
3,Y,com,0.999214
4,Y,amb,0.969242
5,Y,fbp,1.04402
6,Y,rec,1.02558
7,Y,con,0.998728
8,Y,agr,1.02651
9,Y,eec,0.993423


# Create comparison dataframe

In [7]:
df_mcp = DataFrame(CSV.File("./Results/national_sols_julia_mcp.csv",stringtype=String))
    
df = DataFrame(CSV.File("./Results/national_sols_gams.csv",stringtype=String)) |>
    x -> dropmissing(x) |>
    x -> outerjoin(df_mpsge,df_mcp,x, on = [:var,:domain],makeunique=true) |>
    x -> rename(x, :value => :julia_mpsge, :value_1 => :julia_mcp, :value_2 => :gams) |>
    x -> transform(x, [:julia_mpsge,:julia_mcp] => ((a,b) -> abs.(a-b)) => :diff) |>
    #x -> filter(:julia_mcp => y -> !ismissing(y) && y !=0 , x) |>
    x -> sort(x,:diff;rev=true)

Row,var,domain,julia_mpsge,julia_mcp,gams,Column4,Column5,Column6,Column7,diff
,String,String,Float64?,Float64?,Float64?,String?,Float64?,String?,Float64?,Float64?
1,A,fbt,missing,0.0,1.0,"A[(:fbt,)]",1.0,"A[(:fbt,)]",0.0,missing
2,A,mvt,missing,0.0,1.0,"A[(:mvt,)]",1.0,"A[(:mvt,)]",0.0,missing
3,A,gmt,missing,0.0,1.0,"A[(:gmt,)]",1.0,"A[(:gmt,)]",0.0,missing
4,PA,fbt,missing,0.0,1.0,"PA[(:fbt,)]",1.0,"PA[(:fbt,)]",1.0,missing
5,PA,mvt,missing,0.0,1.0,"PA[(:mvt,)]",1.0,"PA[(:mvt,)]",1.0,missing
6,PA,gmt,missing,0.0,1.0,"PA[(:gmt,)]",1.0,"PA[(:gmt,)]",1.0,missing
7,RA,,13775.8,missing,missing,missing,missing,missing,missing,missing
8,PFX,,1.07723,missing,missing,missing,missing,missing,missing,missing
9,PA,trn,1.26369,1.14243,1.14243,"PA[(:trn,)]",1.14243,"PA[(:trn,)]",1.14243,0.121258


In [8]:
df[80:120,:]

Row,var,domain,julia_mpsge,julia_mcp,gams,Column4,Column5,Column6,Column7,diff
,String,String,Float64?,Float64?,Float64?,String?,Float64?,String?,Float64?,Float64?
1,PA,fof,1.06871,0.966165,0.966165,"PA[(:fof,)]",0.966165,"PA[(:fof,)]",0.966165,0.10255
2,PY,mch,1.06848,0.965957,0.965957,"PY[(:mch,)]",0.965957,"PY[(:mch,)]",0.965957,0.102527
3,PA,ore,1.06757,0.965134,0.965134,"PA[(:ore,)]",0.965134,"PA[(:ore,)]",0.965134,0.10244
4,PY,ore,1.06696,0.964581,0.964581,"PY[(:ore,)]",0.964581,"PY[(:ore,)]",0.964581,0.102381
5,PY,nmp,1.06622,0.963908,0.963908,"PY[(:nmp,)]",0.963908,"PY[(:nmp,)]",0.963908,0.10231
6,PY,con,1.06528,0.963062,0.963062,"PY[(:con,)]",0.963062,"PY[(:con,)]",0.963062,0.10222
7,PY,uti,1.06507,0.962873,0.962873,"PY[(:uti,)]",0.962873,"PY[(:uti,)]",0.962873,0.1022
8,PY,otr,1.06503,0.962834,0.962834,"PY[(:otr,)]",0.962834,"PY[(:otr,)]",0.962834,0.102196
9,PY,brd,1.06492,0.962738,0.962738,"PY[(:brd,)]",0.962738,"PY[(:brd,)]",0.962738,0.102186


# Full Julia MPSGE Solution

In [21]:
replace_dict = Dict(
    "Y" => [r"^Y\[\(:(\w{3}),\)]", s"(:\1,)"],
    "A" => [r"^A\[\(:(\w{3}),\)]", s"(:\1,)"],
    "MS" => [r"^MS\[\(:(\w{3}),\)]", s"(:\1,)"],
    "PA" => [r"^PA\[\(:(\w{3}),\)]", s"(:\1,)"],
    "PY" => [r"^PY\[\(:(\w{3}),\)]", s"(:\1,)"],
    "PVA" => [r"^PVA\[\(:(\w{6,7}),\)]", s"(:\1,)"],
    "PM" => [r"^PM\[\(:(\w{3}),\)]", s"(:\1,)"],
    "PFX" => [r"^PFX", s""],
    "RA" => [r"^RA", s""],
    "C" => [r"^PA\[(\w{3})\]ρRA", s"(:\1,)"],
    "S_PY_Y" => [r"^PY\[(\w{3})\]‡Y\[(\w{3})\]", s"(:\1,:\2)"],
    "D_PA_Y" => [r"^PA\[(\w{3})\]†Y\[(\w{3})\]", s"(:\1,:\2)"],
    "D_PVA_Y" => [r"^PVA\[(\w{3})\]†Y→VA(\w{3})", s"(:\1,:\2)"],
    "S_PM_MS" => [r"^PM\[(\w{3})\]‡MS\[(\w{3})\]", s"(:\1,:\2)"],
    "D_PY_MS" => [r"^PY\[(\w{3})\]†MS\[(\w{3})\]", s"(:\1,:\2)"],
    "S_PA_A" => [r"^PA\[(\w{3})\]‡A\[(\w{3})\]", s"(:\1,:\2)"],
    "S_PFX_A" => [r"^PFX‡A\[(\w{3})\]", s"(:\1,)"],
    "D_PY_A" => [r"^PY\[(\w{3})\]†A→dm(\w{3})", s"(:\1,:\2)"],
    "D_PFX_A" => [r"^PFX†A→dm(\w{3})", s"(:\1,)"],
    "D_PM_A" => [r"^PM\[(\w{3})\]†A\[(\w{3})\]", s"(:\1,:\2)"],
    );

In [23]:
d = DataFrame(CSV.File("./Results/FullReportCounter.csv",stringtype=String)) |>
    x -> transform(x, :margin => (y -> ifelse.(y.==".","0",y)) => :margin) |>
    x -> transform(x, :margin => (y -> parse.(Float64,y))=>:margin);

In [24]:

df = DataFrame()

for (var,(reg,sub)) in replace_dict
    df_ = d |>
        x-> filter(:var_name => y -> occursin.(reg ,y),x) |>
        x -> transform(x,
            :var_name => (y -> replace.(y, reg => sub)) => :domain,
            :var_name => (y -> var) => :var
        ) |>
        x -> select(x, [:var_name,:var,:domain,:value,:margin])
    
    df = vcat(df,df_)
end

In [25]:
df

Row,var_name,var,domain,value,margin
,String,String,String,Float64,Float64
1,PA[ppd]†Y[ppd],D_PA_Y,"(:ppd,:ppd)",43.8969,-0.0
2,PA[res]†Y[ppd],D_PA_Y,"(:res,:ppd)",0.162683,-0.0
3,PA[com]†Y[ppd],D_PA_Y,"(:com,:ppd)",0.197119,-0.0
4,PA[fbp]†Y[ppd],D_PA_Y,"(:fbp,:ppd)",1.66646,-0.0
5,PA[rec]†Y[ppd],D_PA_Y,"(:rec,:ppd)",0.00511639,-0.0
6,PA[con]†Y[ppd],D_PA_Y,"(:con,:ppd)",0.668667,-0.0
7,PA[eec]†Y[ppd],D_PA_Y,"(:eec,:ppd)",0.454946,-0.0
8,PA[pub]†Y[ppd],D_PA_Y,"(:pub,:ppd)",0.00411802,-0.0
9,PA[ins]†Y[ppd],D_PA_Y,"(:ins,:ppd)",0.234656,-0.0
